In [1]:
%run common.ipynb

tokenizer.decode(tokenizer.get_data(third_number=False))

decoder = ['P', 'S', 'E', 'U', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '*', '/', '.', ':', '=', '_']
encoder = {'P': 0, 'S': 1, 'E': 2, 'U': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '+': 14, '-': 15, '*': 16, '/': 17, '.': 18, ':': 19, '=': 20, '_': 21}
length check: True
22
cuda


'S-1468.63=84.99*-17.28E'

In [2]:
' 生成 批数据 '
def get_batch_data():
    """
    返回:
        input_ids: 2D Tensor, 每一行代表一个经过 tokenizer 编码后的输入序列, 
                包含由 tokenizer.encoder['P'] 进行的 Padding 填充
        attention_mask: 2D Tensor, 与 input_ids 形状相同, 用来标记哪些位置是实际的输入数据, 哪些是填充的标记
                tokenizer.encoder['P'] 标记位置为 0, 其余为 1,
                模型在计算 attention 时会利用这个 mask 来忽略填充部分, 从而只关注真正有意义的输入数据
    
    保持批次中所有数据的形状一致，便于并行计算
    """

    # 两数计算
    data = [tokenizer.get_data(third_number=False) for i in range(64)]

    #求最大长度
    lens = max([len(i) for i in data])

    #做个白板
    input_ids = torch.full(
        (len(data), lens),
        tokenizer.encoder['P'],
        device=device
    )
    # print(f'input_ids, the template: {input_ids}, and its size: {input_ids.shape}')

    # 往白板里黏贴数据
    for i, d in enumerate(data):
        input_ids[i, :len(d)] = torch.LongTensor(d)  # 将列表 d 转换成 PyTorch 中的长整型张量 LongTensor

    attention_mask = (input_ids != tokenizer.encoder['P']).long()

    return input_ids, attention_mask


input_ids, attention_mask = get_batch_data()
print(input_ids.shape, attention_mask.shape)  # (batch_size, seq_len)

torch.Size([64, 23]) torch.Size([64, 23])


In [4]:
# model_gen = ModelGEN()
model_gen = QwenModel()
optimizer = torch.optim.Adam(model_gen.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.encoder['P'])

for i in range(1000):
    input_ids, attention_mask = get_batch_data()

    out = model_gen(input_ids=input_ids, attention_mask=attention_mask)
    # out -> (batch_size, seq_len, vocab_size) -> torch.Size([64, 23, 22])
    if i % 1000 == 0:
        print(f'out shape: {out.shape}')
        # -> out shape: torch.Size([64, 23, 22])
    
    loss = criterion(
        # 将 模型输出(out) 和 目标序列(input_ids) 做一个时序偏移
        # 模型预测结果 去掉每个序列最后一个时间步的输出, 因为最后一个输出没有对应的下一个 token
        # flatten:  (batch_size, seq_len-1, vocab_size) --> (batch_size * (seq_len - 1), vocab_size)
        out[:, :-1].flatten(end_dim=1),
        # 目标 token 取 input_ids[:, 1:], 把输入序列向右平移一位, 作为每个时间步的真实标签
        # flatten: (batch_size, seq_len-1) --> (batch_size * (seq_len - 1))
        input_ids[:, 1:].flatten()
    )
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 100 == 0:
        gen = generate(model_gen, input_ids[0].unsqueeze(0))
        print(i, tokenizer.decode(gen[0].tolist()))

model_gen.to('cpu')
torch.save(model_gen, 'gen.model')

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /root/autodl-tmp/LLMs/qwen2.5_14b_int8/models--Qwen--Qwen2.5-14B-Instruct-GPTQ-Int8.